In [1]:
import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

import json
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
from Classes.cognitive_model_agents import MODELS
from Classes.cognitive_model_agents import *
from Classes.parameter_recovery import ParameterFit

In [3]:
# Create paths for data and results
data_folder = Path('..', 'data', 'human')
folder_2P = Path('..', 'reports', 'MLE')
folder_2P.mkdir(parents=True, exist_ok=True)
best_fit_file_2P = Path(folder_2P, f'best_fit_2P.json')

folder_all = Path('..', 'reports', 'MLE')
folder_all.mkdir(parents=True, exist_ok=True)
best_fit_file_all = Path(folder_all, f'best_fit_all.json')

## Fit to 2-player data

In [ ]:
#Load 2P data

file_name = '2-player-UR.csv'
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')
data = pd.read_csv(file)
data.head()

In [ ]:
# Create optimization hyperparameters
hyperparameters = {
    'init_points':4,
    'n_iter':8
}

#Initialize records
best_fit = {
    'model_name': list(),
    'deviance': list(),
    'AIC': list(),
    'free_parameters': list()
}

In [ ]:
# MODELS is a dictionary with the free parameters for 
# each model and comes from Classes.cognitive_model_agents
for model_name, model in tqdm(MODELS.items(), desc='Models '):
    # if model_name not in ['MFP-M3']:
    # 	continue
    print(f'Fitting data to model {model_name}...')
    best_fit['model_name'].append(model_name)
    print('Creating parameter recovery class...')
    pf = ParameterFit(
        agent_class=model["class"],
        model_name=model_name,
        free_parameters=model["free_parameters"],
        data=data,
        optimizer_name='bayesian'
    )
    print('Running bayesian optimizer...')
    res = pf.get_optimal_parameters(hyperparameters)
    best_fit['deviance'].append(-res["target"])
    best_fit['AIC'].append(2*len(model["free_parameters"]) - 2*res["target"])
    best_fit['free_parameters'].append(res["params"])
print(best_fit)

In [ ]:
list_lens = [len(val) for key, val in best_fit.items()]
max_len = max(list_lens)
assert(np.all(x == max_len for x in list_lens))
with open(best_fit_file_2P, 'w') as f:
    json.dump(best_fit, f)
print(f'Model recovery data writen to file {best_fit_file_2P}')

## Fit to all player data

In [4]:
# Load data into a dataframe
file_names = [
	# '2-player-UR.csv',
	'3-player-IU.csv',
	'4-player-IU.csv',
	'5-player-IU.csv',
	'6-player-IU.csv',
	'7-player-IU.csv',
	'8-player-IU.csv',
	'9-player-IU.csv',
	'11-player-IU.csv',
	'12-player-IU.csv',
]
df_list = list()
for file_name in file_names:
	file = data_folder / Path(file_name)
	print(f'Loading data from {file}...')
	df = pd.read_csv(file)
	df_list.append(df)
data = pd.concat(df_list, ignore_index=True)
columns = ['threshold', 'num_players', 'group', 'round', 'player', 'score', 'decision']
drop_columns = [col for col in data.columns if col not in columns]
data.drop(columns=drop_columns, inplace=True)
data.head()

Loading data from ../data/human/3-player-IU.csv...
Loading data from ../data/human/4-player-IU.csv...
Loading data from ../data/human/5-player-IU.csv...
Loading data from ../data/human/6-player-IU.csv...
Loading data from ../data/human/7-player-IU.csv...
Loading data from ../data/human/8-player-IU.csv...
Loading data from ../data/human/9-player-IU.csv...
Loading data from ../data/human/11-player-IU.csv...
Loading data from ../data/human/12-player-IU.csv...


,threshold,round,player,decision,score,num_players,group
0,0.333333,1.0,406360458182849,1.0,-1.0,3,17
1,0.333333,1.0,679877000681823,1.0,-1.0,3,17
2,0.333333,1.0,740425726975861,1.0,-1.0,3,17
3,0.333333,2.0,406360458182849,1.0,-1.0,3,17
4,0.333333,2.0,679877000681823,1.0,-1.0,3,17


In [ ]:
# Create optimization hyperparameters
hyperparameters = {
    'init_points':4,
    'n_iter':8
}

#Initialize records
best_fit = {
    'model_name': list(),
    'deviance': list(),
    'num_parameters': list(),
    'AIC': list(),
    'free_parameters': list()
}

# MODELS is a dictionary with the free parameters for 
# each model and comes from Classes.cognitive_model_agents
for model_name, model in tqdm(MODELS.items(), desc='Models '):
    # if model_name not in ['QAttendance', 'QFairness']:
    # 	continue
    print(f'Fitting data to model {model_name}...')
    best_fit['model_name'].append(model_name)
    print('Creating parameter recovery class...')
    pf = ParameterFit(
        agent_class=model["class"],
        model_name=model_name,
        free_parameters=model["free_parameters"],
        data=data,
        optimizer_name='bayesian',
        with_treatment=False
    )
    print('Running bayesian optimizer...')
    res = pf.get_optimal_parameters(hyperparameters)
    best_fit['deviance'].append(-res["target"])
    best_fit['num_parameters'].append(len(model["free_parameters"]))
    best_fit['AIC'].append(2*len(model["free_parameters"]) - 2*res["target"])
    best_fit['free_parameters'].append(res["params"])
print(best_fit)
with open(best_fit_file_all, 'w') as f:
    json.dump(best_fit, f)
print(f'Model recovery data writen to file {best_fit_file_all}')

Models :   0%|          | 0/17 [00:00<?, ?it/s]

Fitting data to model Random...
Creating parameter recovery class...
pbounds: {'go_prob': (0, 1)}
Running bayesian optimizer...
|   iter    |  target   |  go_prob  |
-------------------------------------
| 1         | -2.009e+0 | 0.417     |
| 2         | -1.982e+0 | 0.7203    |
| 3         | -1.098e+0 | 0.0001144 |
| 4         | -2.314e+0 | 0.3023    |
| 5         | -7.919e+0 | 1.0       |
| 6         | -2.007e+0 | 0.7329    |
| 7         | -1.864e+0 | 0.5555    |
| 8         | -1.871e+0 | 0.6233    |
| 9         | -1.909e+0 | 0.4873    |
| 10        | -1.865e+0 | 0.5527    |
| 11        | -2.143e+0 | 0.3574    |
| 12        | -1.861e+0 | 0.5918    |
Fitting data to model WSLS...
Creating parameter recovery class...
pbounds: {'inverse_temperature': (4, 32), 'go_drive': (0, 1), 'wsls_strength': (0, 2)}
Running bayesian optimizer...
|   iter    |  target   | go_drive  | invers... | wsls_s... |
-------------------------------------------------------------
| 1         | -6.28e+04 | 0.417 

In [6]:
list_lens = [len(val) for key, val in best_fit.items()]
max_len = max(list_lens)
assert(np.all(x == max_len for x in list_lens))
with open(best_fit_file_all, 'w') as f:
    json.dump(best_fit, f)
print(f'Model recovery data writen to file {best_fit_file_all}')

Model recovery data writen to file ../reports/MLE/best_fit_all.json
